In [ ]:
import os
import sys

sys.path.insert(0, '../..')

from absl import flags
flags.FLAGS(['notebook'])

import tensorflow as tf

from lmml.models.yolov3 import draw_labels, draw_outputs, YoloV3, yolo_outputs_to_pred, yolo_anchor_masks, yolo_anchors
from lmmlscripts.yolo.dataset import load_dataset

In [ ]:
# Load val dataset
ds = load_dataset('tfds://voc/2007', 'validation', 4, 416)
ds_iter = iter(ds)

In [ ]:
# Load checkpoints
strategy = tf.distribute.MirroredStrategy(devices=tf.config.list_logical_devices('CPU'))
with strategy.scope():
    BACKBONE_SAVED_MODEL_PATH = '../../output/darknet/saved_model/backbone'
    YOLO_SAVED_MODEL_PATH = '../../output/yolo/saved_model/model'
    backbone_model  = (
        tf.saved_model.load(BACKBONE_SAVED_MODEL_PATH)
        .signatures['serving_default'])

    model = (
        tf.saved_model.load(YOLO_SAVED_MODEL_PATH)
        .signatures['serving_default']
    )



In [ ]:
# run checkpoint on dataset and plot the output on image.

@tf.function
def predict(input):
    outputs = backbone_model(input_1=input)
    x_36, x_61, x = outputs['add_10'], outputs['add_18'], outputs['add_22']
    return model(f_36=x_36, f_61=x_61, f=x)

In [ ]:

class_names = ['class_' + str(i) for i in range(100)]

inp, labels = next(ds_iter)
outputs = strategy.run(predict, (inp,))
boxes, scores, classes, nums = yolo_outputs_to_pred(
    outputs["yolo_output_0"], outputs["yolo_output_1"], outputs["yolo_output_2"],
    yolo_anchors, yolo_anchor_masks, 20)

In [ ]:
import cv2
from matplotlib import pyplot as plt
img = cv2.cvtColor(inp[0].numpy(), cv2.COLOR_RGB2BGR)
plt.figure()
plt.imshow(draw_outputs(img, (boxes, scores, classes, nums), class_names))

res = draw_labels(inp[0], labels[0].numpy(), class_names)
plt.figure()
plt.imshow(res)

In [ ]:
labelsi